In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from string import punctuation
import re
import seaborn as sns
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
import time 


df = pd.read_csv('~/ML/NFL/finalSetSmall.csv')
df1 = df.select_dtypes(exclude=['object'])
df2 = df.select_dtypes(['object'])
df1 = df1.replace([np.inf, -np.inf], np.nan)
df1 = df1.replace(np.nan, 10)
yTrain = df1['Yards']
playId = df1['PlayId']
xTrain = df1.drop(['GameId',  'PlayId', 'Yards'], axis=1)
#xTrain = pd.get_dummies(xTrain)



x1 = xTrain.iloc[:23000].to_numpy()
y = yTrain.iloc[:23000].to_numpy()

xTest1 = xTrain.iloc[23000:].to_numpy()
yTest = yTrain.iloc[23000:].to_numpy()

scaler = preprocessing.StandardScaler()
pt = preprocessing.PowerTransformer()

#xNorm = pt.fit_transform(x1)
#xNormTest = pt.transform(xTest1)

xScale = scaler.fit_transform(x1)
xScaleTest = scaler.transform(xTest1)

#poly = PolynomialFeatures(2)
#xPoly = poly.fit_transform(xScale) 

In [2]:
# dfDummies = pd.get_dummies(df2)
# dummies = dfDummies.to_numpy()
# sel = VarianceThreshold(threshold = 0.1)
# dummies2 = sel.fit_transform(dummies)
# dummies2.shape
# dummiesTry = dummies2[:23000]
# dummiesTest = dummies2[23000:]
# xScale = np.concatenate((dummiesTry, xScale), axis=1)
# xScaleTest = np.concatenate((dummiesTest, xScaleTest), axis = 1)

In [3]:
# sel = VarianceThreshold(threshold = 1)
# xScalePrime = sel.fit_transform(xScale)
# xScaleTestPrime = sel.transform(xScaleTest)

# poly = PolynomialFeatures(2)
# xScalePoly = poly.fit_transform(xScalePrime)
# xScaleTestPoly = poly.transform(xScaleTestPrime)

# xScale = xScalePoly
# xScaleTest = xScaleTestPoly

In [4]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

from sklearn import linear_model
reg1 = linear_model.Ridge(alpha=0.001, solver = 'svd')
reg2 = linear_model.Ridge(alpha=0.001, solver = 'cholesky')
reg3 = linear_model.Ridge(alpha=0.001, solver = 'lsqr')

from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier(n_estimators=100, random_state=0)
rf2 = RandomForestClassifier(n_estimators=300, random_state=0)
rf3 = RandomForestClassifier(n_estimators=50, random_state=0)

from sklearn.svm import SVR
svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

from sklearn.neural_network import MLPRegressor
nnR = MLPRegressor(alpha=1e-6, hidden_layer_sizes=(1000,), random_state=1)

from sklearn.neural_network import MLPClassifier
nnC = MLPClassifier(alpha=1e-6, hidden_layer_sizes=(50,), random_state=1)

from sklearn.neighbors import KNeighborsRegressor
KNN = KNeighborsRegressor(n_neighbors=1000)

In [5]:
def gauss_func(mu, sigma, bins): 
    val = 1/(sigma*np.sqrt(2*np.pi))*np.exp(-1*(bins - mu)**2/(2*sigma**2))
    return val
                                            
def gauss(pred):
    predList = []
    sumVal = 0
    for i in range(-99, 100): 
        sumVal += gauss_func(pred, 3, i)
        predList.append(sumVal)
    return predList

def y_cdf(yTest): 
    array = np.zeros((len(yTest), 199))
    for j in range(len(yTest)): 
        for i in range(int(yTest[j])+99):
            array[j][i] = 1
    array = 1-array
    
    return array
   
def score(out, label): 
    score = np.sum(np.power(out-label,2))/(199)
    return score

def mse(yards, prediction):
    mean = ((yards - prediction)*(yards - prediction)).sum()/len(yards)
    return mean

y_test_cdf = np.asarray(y_cdf(yTest))

In [6]:
beginTime = time.time()
reg1.fit(xScale, y)
rf1.fit(xScale, y)
#svr.fit(xScale, y)
nnR.fit(xScale, y)
#reg3.fit(xScale, y)
print(time.time() - beginTime)

384.2779870033264


In [ ]:
# beginTime = time.time()
# nnR.fit(xScale, y)
# print(time.time() - beginTime)

In [7]:
regPreds = [reg1.predict(xScaleTest), rf1.predict(xScaleTest), svr.predict(xScaleTest), 
           nnR.predict(xScale, y)]

for j in range(len(regPreds)):
    scores = []
    for i in range(len(yTest)): 
        scores.append(score(y_test_cdf[i], gauss(regPreds[j][i])))
    scores = np.asarray(scores)
    print(scores.mean())

NotFittedError: This SVR instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
def pipeline(data_frame, estimators, params, filename, poly_fit):

    print("here: 1")
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(filename=filename, filemode='w', level=logging.INFO)
    logger = logging.getLogger()
    
    minimal_error, best_estimator = None, None
    
    for estimator in estimators:
        
        try: 
            print(f"Starting with estimator: {estimator.__name__}")
            logging.info(f"Starting with estimator: {estimator.__name__}")

            for index, cur_params in enumerate(params[estimator.__name__]):
                print(cur_params)
                regressor = estimator(**cur_params)
                print("here: 2")

                # To collect MSE over each split
                errors = []

                # to collect accuracies
                accuracies = []
                top_1_accs = []

                #validation_data = data_frame
                #validation_data = validation_data.sort_values(by='points_won', ascending=False)

                # Get train data
                train_x = XTrain
                train_y = yTrain
                print("here: 3")
                
                # Validate over one season only
                val_x = XVal
                val_y = yVal
                #val_y = val_y.reshape(val_y.shape[0], )
                print("here: 4")
                

                if poly_fit is not None:
                    train_x = poly_fit.fit_transform(train_x)
                    val_x = poly_fit.fit_transform(val_x)

                
                shuffle_x, shuffle_y = shuffle(train_x, train_y)
                

                regressor.fit(shuffle_x, shuffle_y)
                predicted_y = regressor.predict(val_x)

                sorted_indices = np.argsort(predicted_y)[::-1]
                correct_indices = np.arange(len(val_y))

                curr_error = mean_squared_error(val_y, predicted_y)
                errors.append(curr_error)
                mean_error = np.average(errors)
            
                logging.info(f"Params: {cur_params}, MSE over all splits is: {mean_error:.4f}")
                print(
                    f"Params: {cur_params}, MSE over all splits is: {mean_error:.4f}")

                if minimal_error is None or mean_error < minimal_error:
                    minimal_error = mean_error
                    best_estimator = estimator(**cur_params)
        except Exception:
            print(f"Exception: {estimator}")
            continue
            
    return best_estimator

estimators = [Ridge]

params = {Ridge.__name__: 
    [
        {
            'alpha': 1.0, 'solver':'svd'
        },
        {
            'alpha': 10.0, 'solver':'svd'
        },
        {
            'alpha': 30.0, 'solver':'svd'
        },
        {
            'alpha': 50.0, 'solver':'svd'
        },
    ],
         }

best_estimator = pipeline(
    data_frame=val_data,
    estimators=estimators,
    params=params,
    filename="log_reg_poly_2.txt",
    # scaler=MinMaxScaler(),
    poly_fit=PolynomialFeatures(degree=2, interaction_only=True),
)